## XGB on modified data

#### Imports

In [1]:
import pandas as pd
import numpy as np
import copy
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cross_validation import cross_val_score
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from xgboost import plot_importance
from matplotlib import pyplot as plt
import xgboost as xgb
import gc
import sys


/home/matia_alexander/miniconda3/envs/deeplearning/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/matia_alexander/miniconda3/envs/deeplearning/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/matia_alexander/miniconda3/envs/deeplearning/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/matia_alexander/miniconda3/envs/deeplearning/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/matia_alexander/miniconda3/envs/deeplearning/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: Depre

### Parameters:

In [2]:

run_title = "XGB_Full_w_Feature_Extraction_top1000"

Word_Notes = "added bureau and prev app features, Weights v2.0"
rounds = 2000

par = {  'eta': .01,
         'gamma': 6.3324,
         'max_depth': 29,
         'min_child_weight': 19,
          'colsample_byree': .8315,
          'objective': 'binary:logistic',
          'eval_metric': 'auc',
          'n_job': 16,
          'booster': 'gbtree',
           'max_delta_step':1.3239,
           'subsample':.995
      }

In [3]:
Output_Name = run_title + ".csv"

### Data

In [4]:
#Not reading in Test yet to avoid memory issues
data_path = "/home/matia_alexander/data/"
output_path = "/home/matia_alexander/output/"
application_train = pd.read_csv(data_path + "train_modified.csv")
application_test = pd.read_csv(data_path + "test_modified.csv")
sample_submission = pd.read_csv(data_path + 'sample_submission.csv')

## Feature Selection

In [5]:
#Test result of keeping only MEDIAN imputed columns when median-imputed columns exist

Feature_Set = [i for i in application_train.columns]
Feature_Set = [i for i in application_train.columns 
              if '.trmean' not in i
              if '.trmode' not in i]
Feature_Set = [i for i in Feature_Set
              if not (i + '.trmedian') in Feature_Set]

Feature_Set_test = [i for i in Feature_Set
                   if i not in ['TARGET', 'fold', 'Weights']]

In [6]:
application_train = application_train.loc[:,Feature_Set]
application_test = application_test.loc[:, Feature_Set_test]

In [7]:
non_numeric_cols = []
for i in application_train.columns:
    if application_train[i].dtype.name == "object":
        non_numeric_cols.append(i)

#one-hot encode categoricals in train and test sets

#Build dataset of only non-numerics
application_train_non_numeric = application_train.loc[:,non_numeric_cols]
#Remove non-encoded variables from original dataset
application_train.drop(columns = non_numeric_cols, inplace = True)
#Convert non-numerics into dummies/ one-hot encode
application_train_non_numeric_dummies = pd.get_dummies(application_train_non_numeric)
#Join encoded dataset back with original
application_train = pd.concat([application_train, application_train_non_numeric_dummies], axis = 1)


non_numeric_cols = []
for i in application_test.columns:
    if application_test[i].dtype.name == "object":
        non_numeric_cols.append(i)

#one-hot encode categoricals in train and test sets

#Build dataset of only non-numerics
application_test_non_numeric = application_test.loc[:,non_numeric_cols]
#Remove non-encoded variables from original dataset
application_test.drop(columns = non_numeric_cols, inplace = True)
#Convert non-numerics into dummies/ one-hot encode
application_test_non_numeric_dummies = pd.get_dummies(application_test_non_numeric)
#Join encoded dataset back with original
application_test = pd.concat([application_test, application_test_non_numeric_dummies], axis = 1)




In [8]:
train_cols = [i for i in application_train.columns]
test_cols = [i for i in application_test.columns]
train_not_test = [i for i in train_cols if i not in test_cols]
test_not_train = [i for i in test_cols if i not in train_cols]

print("Columns in Train but not Test: \n" + str(train_not_test))
print("Columns in Test but not Train: \n" + str(test_not_train))

Columns in Train but not Test: 
['TARGET', 'fold', 'Weights', 'CODE_GENDER_XNA', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown']
Columns in Test but not Train: 
[]


In [9]:
#Dropping one-hot encoded column expansions that resulted from unique categories
application_train.drop(columns = ["CODE_GENDER_XNA", "NAME_INCOME_TYPE_Maternity leave",
                                 "NAME_FAMILY_STATUS_Unknown"], inplace = True)

### Select most important features

In [10]:
Extracted_features = ['Avg_EXT_SOURCE_application',
 'ANNUITY_LENGTH',
 'MAX_DAYS_CREDIT_bureau',
 'CNT_INSTALMENT_FUTURE_min_sd_POS_CASH',
 'MAX_DAYS_CREDIT_ENDDATE_bureau',
 'DAYS_BIRTH',
 'Refused_count_NAME_CONTRACT_STATUS_prev_app',
 'CODE_GENDER_M',
 'MIN_AMT_CREDIT_SUM_bureau',
 'AMT_ANNUITY_AMT_GOODS_PRICE_ratio_application',
 'DAYS_BIRTH_EXT_SOURCE_1_ratio_application',
 'MEAN_AMT_CREDIT_SUM_DEBT_bureau',
 'MEAN_AMT_CREDIT_SUM_bureau',
 'EXT_SOURCE_1_DAYS_ID_PUBLISH_ratio_application',
 'CREDIT_TO_GOODS_RATIO',
 'AMT_ANNUITY_EXT_SOURCE_1_ratio_application',
 'AMT_PAYMENT_min_sum_installments_payments',
 'SD_DAYS_CREDIT_ENDDATE_bureau',
 'CNT_FAM_MEMBERS_EXT_SOURCE_3_ratio_application',
 'DAYS_BIRTH_EXT_SOURCE_3_ratio_application',
 'SD_AMT_CREDIT_SUM_DEBT_bureau',
 'CNT_INSTALMENT_FUTURE_Mode_sd_POS_CASH',
 'MIN_Delta_Credit_prev_app',
 'MEAN_DAYS_ENDDATE_FACT_bureau',
 'AMT_PAYMENT_n_distinct_mean_installments_payments',
 'AMT_ANNUITY_EXT_SOURCE_3_ratio_application',
 'SD_AMT_ANNUITY_bureau',
 'SOURCES_PROD',
 'INCOME_CREDIT_PERC',
 'high_count_NAME_YIELD_GROUP_prev_app',
 'DAYS_EMPLOYED_PERC',
 'REGION_RATING_CLIENT_W_CITY',
 'MEAN_DAYS_CREDIT_bureau',
 'MAX_AMT_CREDIT_SUM_DEBT_bureau',
 'SD_DAYS_DECISION_prev_app',
 '0_count_bbalance',
 'AMT_PAYMENT_min_max_installments_payments',
 'AMT_ANNUITY',
 'CNT_INSTALMENT_FUTURE_mean_median_POS_CASH',
 'Day_Diff_installments_min_median_installments_payments',
 'Day_Diff_installments_min_min_installments_payments',
 'CNT_INSTALMENT_FUTURE_mean_max_POS_CASH',
 'SUM_AMT_CREDIT_SUM_DEBT_bureau',
 'CNT_INSTALMENT_FUTURE_median_sd_POS_CASH',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'MAX_AMT_CREDIT_SUM_bureau',
 'CNT_INSTALMENT_FUTURE_median_mean_POS_CASH',
 'AMT_DRAWINGS_ATM_CURRENT_mean_mean_credit_card_balance',
 'Day_Diff_installments_min_sum_installments_payments',
 '1_proportion_bbalance',
 'DAYS_LAST_PHONE_CHANGE_AMT_ANNUITY_ratio_application',
 'EXT_SOURCE_2.x',
 'MEAN_AMT_ANNUITY_bureau',
 'MEAN_AMT_DOWN_PAYMENT_prev_app',
 'OWN_CAR_AGE_EXT_SOURCE_3_ratio_application',
 'AMT_CREDIT_DAYS_EMPLOYED_ratio_application',
 'DAYS_LAST_PHONE_CHANGE_DEF_30_CNT_SOCIAL_CIRCLE_ratio_application',
 'OWN_CAR_AGE_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'AMT_ANNUITY_DAYS_EMPLOYED_ratio_application',
 'Payment_Diff_installments_mean_sum_installments_payments',
 'DAYS_BIRTH_DAYS_ID_PUBLISH_ratio_application',
 'MEAN_DAYS_CREDIT_ENDDATE_bureau',
 'SUM_DAYS_LAST_DUE_1ST_VERSION_prev_app',
 'AMT_CREDIT_DAYS_BIRTH_ratio_application',
 'AMT_INSTALMENT_min_mean_installments_payments',
 'DAYS_EMPLOYED_EXT_SOURCE_1_ratio_application',
 'MEAN_AMT_CREDIT_MAX_OVERDUE_bureau',
 'OWN_CAR_AGE_EXT_SOURCE_1_ratio_application',
 'AMT_GOODS_PRICE_AMT_CREDIT_ratio_application',
 'Bureau_Current_Credit_Count',
 'MAX_DAYS_ENDDATE_FACT_bureau',
 'walk-in_count_NAME_PRODUCT_TYPE_prev_app',
 'AMT_INCOME_TOTAL_DAYS_BIRTH_ratio_application',
 'AMT_PAYMENT_max_min_installments_payments',
 'AMT_PAYMENT_min_median_installments_payments',
 'CODE_GENDER_F',
 'DAYS_ID_PUBLISH_EXT_SOURCE_1_ratio_application',
 'MIN_DAYS_CREDIT_UPDATE_bureau',
 'AMT_PAYMENT_CURRENT_median_max_credit_card_balance',
 'C_count_bbalance',
 'Day_Diff_installments_sum_sum_installments_payments',
 'NUM_INSTALMENT_NUMBER_sd_median_installments_payments',
 'Payment_Diff_installments_mean_median_installments_payments',
 'SD_CNT_PAYMENT_prev_app',
 'Carloan_count_CREDIT_TYPE_bureau',
 'X_proportion_bbalance',
 'AMT_INSTALMENT_min_median_installments_payments',
 'AMT_REQ_CREDIT_BUREAU_QRT_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'Day_Diff_installments_Mode_sd_installments_payments',
 'AMT_PAYMENT_max_Mode_installments_payments',
 'AMT_REQ_CREDIT_BUREAU_QRT_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'CNT_INSTALMENT_FUTURE_min_mean_POS_CASH',
 'MEAN_DAYS_CREDIT_UPDATE_bureau',
 'AMT_GOODS_PRICE_EXT_SOURCE_2_ratio_application',
 'Day_Diff_installments_min_max_installments_payments',
 'HC_count_CODE_REJECT_REASON_prev_app',
 'SUM_AMT_CREDIT_SUM_bureau',
 '0_proportion_bbalance',
 'MAX_AMT_DOWN_PAYMENT_prev_app',
 'Payment_Diff_installments_mean_mean_installments_payments',
 'ANNUITY_INCOME_PERC',
 'CNT_DRAWINGS_ATM_CURRENT_mean_max_credit_card_balance',
 'DAYS_LAST_PHONE_CHANGE_DAYS_ID_PUBLISH_ratio_application',
 'NAME_EDUCATION_TYPE_Higher education',
 'SD_DAYS_CREDIT_UPDATE_bureau',
 'Avg_Time_Btwn_Credit',
 'CNT_FAM_MEMBERS_DAYS_EMPLOYED_ratio_application',
 'DAYS_ID_PUBLISH_DAYS_BIRTH_ratio_application',
 'DAYS_LAST_PHONE_CHANGE_EXT_SOURCE_1_ratio_application',
 'EXT_SOURCE_3_DAYS_EMPLOYED_ratio_application',
 'AMT_ANNUITY_EXT_SOURCE_2_ratio_application',
 'AMT_INSTALMENT_min_Mode_installments_payments',
 'MAX_AMT_ANNUITY_bureau',
 'Payment_Diff_installments_mean_min_installments_payments',
 'SD_EXT_SOURCE_application',
 'FRIDAY_count_WEEKDAY_APPR_PROCESS_START_prev_app',
 'NUM_INSTALMENT_VERSION_mean_mean_installments_payments',
 'X_count_bbalance',
 'AMT_GOODS_PRICE_EXT_SOURCE_3_ratio_application',
 'DAYS_LAST_PHONE_CHANGE_EXT_SOURCE_3_ratio_application',
 'AMT_BALANCE_mean_mean_credit_card_balance',
 'AMT_CREDIT_CNT_FAM_MEMBERS_ratio_application',
 'AMT_DRAWINGS_ATM_CURRENT_mean_max_credit_card_balance',
 'AMT_RECIVABLE_sd_min_credit_card_balance',
 'CNT_FAM_MEMBERS_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'MIN_AMT_ANNUITY_prev_app',
 'SDev_Time_Btwn_Cred',
 'AMT_CREDIT_DAYS_ID_PUBLISH_ratio_application',
 'AMT_INCOME_TOTAL_EXT_SOURCE_3_ratio_application',
 'AMT_PAYMENT_sd_Mode_installments_payments',
 'DAYS_ID_PUBLISH_EXT_SOURCE_3_ratio_application',
 'DAYS_LAST_PHONE_CHANGE_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'Day_Diff_installments_sum_min_installments_payments',
 'EXT_SOURCE_1_DAYS_EMPLOYED_ratio_application',
 'EXT_SOURCE_3.x',
 'PHONE_TO_BIRTH_RATIO',
 'AMT_CREDIT_EXT_SOURCE_2_ratio_application',
 'AMT_CREDIT_EXT_SOURCE_3_ratio_application',
 'AMT_INCOME_TOTAL_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'AMT_PAYMENT_sum_max_installments_payments',
 'APARTMENTS_AVG_DEF_30_CNT_SOCIAL_CIRCLE_ratio_application',
 'MONTHS_BALANCE_min_sd_POS_CASH',
 'AMT_ANNUITY_DAYS_BIRTH_ratio_application',
 'EXT_SOURCE_3_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'FLAG_WORK_PHONE',
 'MAX_DAYS_DECISION_prev_app',
 'SUM_DAYS_CREDIT_ENDDATE_bureau',
 'SUM_DAYS_CREDIT_UPDATE_bureau',
 'AMT_PAYMENT_sd_min_installments_payments',
 'CNT_INSTALMENT_FUTURE_mean_mean_POS_CASH',
 'DAYS_EMPLOYED.y',
 'DAYS_ENTRY_PAYMENT_max_max_installments_payments',
 'EXT_SOURCE_3_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'MIN_AMT_GOODS_PRICE_prev_app',
 'SD_DAYS_FIRST_DUE_prev_app',
 'AMT_PAYMENT_sum_sd_installments_payments',
 'Day_Diff_installments_min_mean_installments_payments',
 'EXT_SOURCE_1.x',
 'MIN_DAYS_CREDIT_bureau',
 'NUM_INSTALMENT_VERSION_mean_max_installments_payments',
 'Payment_Diff_installments_sum_median_installments_payments',
 'SD_AMT_CREDIT_SUM_bureau',
 'SD_AMT_DOWN_PAYMENT_prev_app',
 'AMT_GOODS_PRICE_DAYS_EMPLOYED_ratio_application',
 'AMT_INCOME_TOTAL_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'AMT_INCOME_TOTAL_DAYS_EMPLOYED_ratio_application',
 'AMT_PAYMENT_min_min_installments_payments',
 'CashStreet:high_count_PRODUCT_COMBINATION_prev_app',
 'DAYS_ID_PUBLISH_AMT_CREDIT_ratio_application',
 'EXT_SOURCE_min_application',
 'MAX_AMT_CREDIT_MAX_OVERDUE_bureau',
 'MEAN_Delta_Credit_prev_app',
 'SUM_AMT_DOWN_PAYMENT_prev_app',
 'Active_count_CREDIT_ACTIVE_bureau',
 'CNT_CHILDREN_DAYS_EMPLOYED_ratio_application',
 'DAYS_ID_PUBLISH_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'EXT_SOURCE_2_DAYS_EMPLOYED_ratio_application',
 'NUM_INSTALMENT_VERSION_sd_sum_installments_payments',
 'SD_AMT_GOODS_PRICE_prev_app',
 'AMT_ANNUITY_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'AMT_CREDIT_DAYS_REGISTRATION_ratio_application',
 'AMT_INCOME_TOTAL_EXT_SOURCE_1_ratio_application',
 'APARTMENTS_AVG_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'CNT_INSTALMENT_FUTURE_Mode_mean_POS_CASH',
 'DAYS_LAST_PHONE_CHANGE_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'DAYS_REGISTRATION_EXT_SOURCE_3_ratio_application',
 'Day_Diff_installments_mean_min_installments_payments',
 'Day_Diff_installments_sd_min_installments_payments',
 'Day_Diff_installments_sum_Mode_installments_payments',
 'EXT_SOURCE_2_EXT_SOURCE_3_ratio_application',
 'EXT_SOURCE_3_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'MONTHS_BALANCE_Mode_max_POS_CASH',
 'SD_AMT_CREDIT_prev_app',
 'SUM_AMT_CREDIT_MAX_OVERDUE_bureau',
 'AMT_GOODS_PRICE_DAYS_REGISTRATION_ratio_application',
 'AMT_INSTALMENT_min_min_installments_payments',
 'CNT_CHILDREN_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'DAYS_EMPLOYED_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'DAYS_INSTALMENT_sum_min_installments_payments',
 'DAYS_LAST_PHONE_CHANGE_OWN_CAR_AGE_ratio_application',
 'MAX_RATE_DOWN_PAYMENT_prev_app',
 'MEAN_AMT_APPLICATION_prev_app',
 'SD_DAYS_LAST_DUE_prev_app',
 'AMT_INSTALMENT_min_max_installments_payments',
 'AMT_PAYMENT_min_mean_installments_payments',
 'Closed_count_CREDIT_ACTIVE_bureau',
 'DAYS_LAST_PHONE_CHANGE_AMT_GOODS_PRICE_ratio_application',
 'DAYS_REGISTRATION_EXT_SOURCE_1_ratio_application',
 'EXT_SOURCE_3_DAYS_ID_PUBLISH_ratio_application',
 'EXT_SOURCE_max_application',
 'OWN_CAR_AGE_EXT_SOURCE_2_ratio_application',
 'AMT_GOODS_PRICE',
 'AMT_INSTALMENT_max_Mode_installments_payments',
 'CAR_TO_EMPLOY_RATIO',
 'CNT_FAM_MEMBERS_EXT_SOURCE_1_ratio_application',
 'CNT_INSTALMENT_FUTURE_mean_sd_POS_CASH',
 'CNT_INSTALMENT_FUTURE_min_max_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sum_median_POS_CASH',
 'C_proportion_bbalance',
 'DAYS_INSTALMENT_Mode_sd_installments_payments',
 'DAYS_INSTALMENT_sd_sd_installments_payments',
 'Day_Diff_installments_median_sum_installments_payments',
 'Payment_Diff_installments_sum_max_installments_payments',
 'SD_Delta_Credit_prev_app',
 'SUM_Delta_Credit_prev_app',
 'SUM_RATE_DOWN_PAYMENT_prev_app',
 'AMT_INST_MIN_REGULARITY_sd_min_credit_card_balance',
 'AMT_PAYMENT_min_Mode_installments_payments',
 'AMT_PAYMENT_sum_Mode_installments_payments',
 'AMT_RECEIVABLE_PRINCIPAL_median_mean_credit_card_balance',
 'DAYS_BIRTH_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'DAYS_ENTRY_PAYMENT_sd_max_installments_payments',
 'DAYS_REGISTRATION',
 'DAYS_REGISTRATION_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'EXT_SOURCE_1_EXT_SOURCE_2_ratio_application',
 'New_count_NAME_CLIENT_TYPE_prev_app',
 'Payment_Diff_installments_sum_mean_installments_payments',
 'SK_DPD_sd_Mode_POS_CASH',
 'AMT_DRAWINGS_ATM_CURRENT_mean_sum_credit_card_balance',
 'AMT_PAYMENT_sd_sd_installments_payments',
 'AMT_REQ_CREDIT_BUREAU_YEAR_DAYS_EMPLOYED_ratio_application',
 'AMT_REQ_CREDIT_BUREAU_YEAR_DAYS_ID_PUBLISH_ratio_application',
 'CNT_CHILDREN_EXT_SOURCE_1_ratio_application',
 'CNT_DRAWINGS_ATM_CURRENT_mean_mean_credit_card_balance',
 'CNT_INSTALMENT_FUTURE_mean_min_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sd_median_POS_CASH',
 'DAYS_BIRTH_CNT_FAM_MEMBERS_ratio_application',
 'DAYS_ID_PUBLISH_DAYS_EMPLOYED_ratio_application',
 'DAYS_INSTALMENT_median_median_installments_payments',
 'DEF_30_CNT_SOCIAL_CIRCLE_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'MONTHS_BALANCE_mean_sd_POS_CASH',
 'NAME_CONTRACT_STATUS_sum_mean_POS_CASH',
 'NAME_FAMILY_STATUS_Civil marriage',
 'NUM_INSTALMENT_VERSION_sd_sd_installments_payments',
 'Payment_Diff_installments_max_median_installments_payments',
 'SD_DAYS_TERMINATION_prev_app',
 'TOTALAREA_MODE',
 'AMT_PAYMENT_Mode_max_installments_payments',
 'AMT_PAYMENT_median_mean_installments_payments',
 'AMT_REQ_CREDIT_BUREAU_YEAR_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'CNT_FAM_MEMBERS_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'CNT_FAM_MEMBERS_EXT_SOURCE_2_ratio_application',
 'CNT_INSTALMENT_FUTURE_median_sum_POS_CASH',
 'CNT_INSTALMENT_sd_sd_POS_CASH',
 'DAYS_EMPLOYED_AMT_INCOME_TOTAL_ratio_application',
 'DAYS_EMPLOYED_EXT_SOURCE_3_ratio_application',
 'DAYS_ID_PUBLISH',
 'DAYS_ID_PUBLISH_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'Day_Diff_installments_median_sd_installments_payments',
 'MEAN_AMT_GOODS_PRICE_prev_app',
 'MONTHS_BALANCE_sum_sum_POS_CASH',
 'SD_DAYS_CREDIT_bureau',
 'low_normal_count_NAME_YIELD_GROUP_prev_app',
 'AMT_ANNUITY_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'AMT_ANNUITY_DAYS_ID_PUBLISH_ratio_application',
 'AMT_CREDIT_EXT_SOURCE_1_ratio_application',
 'AMT_GOODS_PRICE_DAYS_BIRTH_ratio_application',
 'AMT_PAYMENT_sum_sum_installments_payments',
 'CNT_DRAWINGS_ATM_CURRENT_mean_sum_credit_card_balance',
 'CNT_INSTALMENT_FUTURE_sum_min_POS_CASH',
 'CNT_INSTALMENT_mean_sd_POS_CASH',
 'CNT_INSTALMENT_sd_mean_POS_CASH',
 'CNT_INSTALMENT_sum_mean_POS_CASH',
 'DAYS_ENTRY_PAYMENT_sd_Mode_installments_payments',
 'DAYS_INSTALMENT_max_min_installments_payments',
 'DAYS_REGISTRATION_OWN_CAR_AGE_ratio_application',
 'EXT_SOURCE_1_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'MAX_DAYS_CREDIT_UPDATE_bureau',
 'MIN_DAYS_ENDDATE_FACT_bureau',
 'MONTHS_BALANCE_sum_min_POS_CASH',
 'Payment_Diff_installments_median_median_installments_payments',
 'SD_AMT_APPLICATION_prev_app',
 'SD_AMT_CREDIT_MAX_OVERDUE_bureau',
 'SUM_AMT_ANNUITY_bureau',
 'SUM_DAYS_CREDIT_bureau',
 'AMT_ANNUITY_APARTMENTS_AVG_ratio_application',
 'AMT_CREDIT_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'AMT_PAYMENT_median_max_installments_payments',
 'AMT_PAYMENT_sum_median_installments_payments',
 'CNT_INSTALMENT_FUTURE_Mode_max_POS_CASH',
 'CNT_INSTALMENT_FUTURE_n_distinct_sd_POS_CASH',
 'CNT_INSTALMENT_mean_median_POS_CASH',
 'CNT_INSTALMENT_sd_sum_POS_CASH',
 'DAYS_EMPLOYED_AMT_ANNUITY_ratio_application',
 'MEAN_AMT_ANNUITY_prev_app',
 'MEAN_DAYS_DECISION_prev_app',
 'MONTHS_BALANCE_sum_max_POS_CASH',
 'NUM_INSTALMENT_NUMBER_sd_mean_installments_payments',
 'SUM_DAYS_DECISION_prev_app',
 'XNA_count_NAME_PRODUCT_TYPE_prev_app',
 'AMT_INSTALMENT_mean_Mode_installments_payments',
 'AMT_INSTALMENT_sd_Mode_installments_payments',
 'AMT_PAYMENT_mean_Mode_installments_payments',
 'AMT_PAYMENT_median_Mode_installments_payments',
 'CNT_DRAWINGS_CURRENT_mean_mean_credit_card_balance',
 'CNT_INSTALMENT_FUTURE_n_distinct_mean_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sd_sd_POS_CASH',
 'CNT_INSTALMENT_sum_min_POS_CASH',
 'DAYS_BIRTH_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'DAYS_BIRTH_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'DAYS_EMPLOYED_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'DAYS_ENTRY_PAYMENT_median_sd_installments_payments',
 'DAYS_REGISTRATION_DAYS_EMPLOYED_ratio_application',
 'Day_Diff_installments_sd_Mode_installments_payments',
 'Day_Diff_installments_sum_median_installments_payments',
 'EXT_SOURCE_1_EXT_SOURCE_3_ratio_application',
 'EXT_SOURCE_3_OBS_30_CNT_SOCIAL_CIRCLE_ratio_application',
 'MONTHS_BALANCE_n_distinct_sd_POS_CASH',
 'PHONE_TO_EMPLOY_RATIO',
 'Time_Since_Status_X_bbalance',
 'AMT_ANNUITY_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'AMT_CREDIT',
 'DAYS_ID_PUBLISH_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'DAYS_INSTALMENT_sum_max_installments_payments',
 'EXT_SOURCE_3_APARTMENTS_AVG_ratio_application',
 'LIVINGAREA_MODE',
 'NAME_CONTRACT_STATUS_sd_sd_POS_CASH',
 'SD_DAYS_ENDDATE_FACT_bureau',
 'SD_SELLERPLACE_AREA_prev_app',
 'AMT_GOODS_PRICE_CNT_FAM_MEMBERS_ratio_application',
 'AMT_PAYMENT_TOTAL_CURRENT_Mode_min_credit_card_balance',
 'CNT_INSTALMENT_FUTURE_median_median_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sum_sd_POS_CASH',
 'CNT_INSTALMENT_min_sd_POS_CASH',
 'CNT_INSTALMENT_sd_max_POS_CASH',
 'COMMONAREA_AVG',
 'COMMONAREA_MEDI',
 'Creditcard_count_CREDIT_TYPE_bureau',
 'DAYS_ENTRY_PAYMENT_Mode_max_installments_payments',
 'DAYS_ENTRY_PAYMENT_sd_sum_installments_payments',
 'DAYS_ENTRY_PAYMENT_sum_Mode_installments_payments',
 'DAYS_LAST_PHONE_CHANGE_AMT_INCOME_TOTAL_ratio_application',
 'DAYS_REGISTRATION_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'LIMIT_count_CODE_REJECT_REASON_prev_app',
 'LIVINGAREA_AVG',
 'MEAN_SELLERPLACE_AREA_prev_app',
 'MIN_DAYS_CREDIT_ENDDATE_bureau',
 'NUM_INSTALMENT_VERSION_sd_max_installments_payments',
 'OWN_CAR_AGE_DAYS_EMPLOYED_ratio_application',
 'AMT_GOODS_PRICE_EXT_SOURCE_1_ratio_application',
 'AMT_INCOME_TOTAL_AMT_GOODS_PRICE_ratio_application',
 'AMT_INCOME_TOTAL_DAYS_ID_PUBLISH_ratio_application',
 'AMT_INSTALMENT_median_mean_installments_payments',
 'AMT_INSTALMENT_sum_mean_installments_payments',
 'AMT_INSTALMENT_sum_sd_installments_payments',
 'AMT_INSTALMENT_sum_sum_installments_payments',
 'CNT_INSTALMENT_FUTURE_Mode_median_POS_CASH',
 'CashX-Sell:low_count_PRODUCT_COMBINATION_prev_app',
 'Consumercredit_count_CREDIT_TYPE_bureau',
 'DAYS_EMPLOYED.x',
 'DAYS_ENTRY_PAYMENT_n_distinct_sd_installments_payments',
 'DAYS_ENTRY_PAYMENT_sd_min_installments_payments',
 'DAYS_ID_PUBLISH_AMT_GOODS_PRICE_ratio_application',
 'DAYS_ID_PUBLISH_APARTMENTS_AVG_ratio_application',
 'DAYS_ID_PUBLISH_EXT_SOURCE_2_ratio_application',
 'DAYS_INSTALMENT_Mode_min_installments_payments',
 'DAYS_LAST_PHONE_CHANGE_AMT_CREDIT_ratio_application',
 'DAYS_LAST_PHONE_CHANGE_DAYS_REGISTRATION_ratio_application',
 'Day_Diff_installments_mean_Mode_installments_payments',
 'EXT_SOURCE_2_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'INCOME_PER_PERSON',
 'MAX_AMT_APPLICATION_prev_app',
 'MAX_DAYS_FIRST_DUE_prev_app',
 'MAX_DAYS_LAST_DUE_1ST_VERSION_prev_app',
 'MEAN_RATE_DOWN_PAYMENT_prev_app',
 'MONTHS_BALANCE_Mode_sd_POS_CASH',
 'NAME_CONTRACT_STATUS_mean_min_POS_CASH',
 'NAME_CONTRACT_STATUS_sd_mean_POS_CASH',
 'OBS_60_CNT_SOCIAL_CIRCLE_DAYS_EMPLOYED_ratio_application',
 'SUM_SELLERPLACE_AREA_prev_app',
 'AMT_ANNUITY_OWN_CAR_AGE_ratio_application',
 'AMT_CREDIT_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'AMT_INSTALMENT_mean_min_installments_payments',
 'AMT_PAYMENT_mean_median_installments_payments',
 'CNT_INSTALMENT_FUTURE_mean_Mode_POS_CASH',
 'CNT_INSTALMENT_FUTURE_median_max_POS_CASH',
 'CNT_INSTALMENT_mean_min_POS_CASH',
 'DAYS_BIRTH_EXT_SOURCE_2_ratio_application',
 'DAYS_ENTRY_PAYMENT_sum_max_installments_payments',
 'DAYS_INSTALMENT_Mode_Mode_installments_payments',
 'DAYS_LAST_PHONE_CHANGE',
 'Day_Diff_installments_Mode_min_installments_payments',
 'Day_Diff_installments_mean_mean_installments_payments',
 'Day_Diff_installments_sd_median_installments_payments',
 'Day_Diff_installments_sd_sd_installments_payments',
 'Day_Diff_installments_sd_sum_installments_payments',
 'EXT_SOURCE_2_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'Microloan_count_CREDIT_TYPE_bureau',
 'NUM_INSTALMENT_NUMBER_sd_Mode_installments_payments',
 'OBS_60_CNT_SOCIAL_CIRCLE_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'OWN_CAR_AGE_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'Payment_Diff_installments_max_mean_installments_payments',
 'Payment_Diff_installments_sum_sum_installments_payments',
 'SK_DPD_DEF_max_sd_POS_CASH',
 'SK_DPD_max_median_POS_CASH',
 'AMT_INSTALMENT_sd_median_installments_payments',
 'AMT_INSTALMENT_sum_median_installments_payments',
 'AMT_PAYMENT_median_median_installments_payments',
 'AMT_PAYMENT_n_distinct_sd_installments_payments',
 'AMT_REQ_CREDIT_BUREAU_QRT_DAYS_EMPLOYED_ratio_application',
 'CNT_CHILDREN_EXT_SOURCE_3_ratio_application',
 'CNT_DRAWINGS_CURRENT_sd_min_credit_card_balance',
 'CNT_INSTALMENT_FUTURE_mean_sum_POS_CASH',
 'DAYS_EMPLOYED_APARTMENTS_AVG_ratio_application',
 'DAYS_ENTRY_PAYMENT_sum_min_installments_payments',
 'DAYS_LAST_PHONE_CHANGE_EXT_SOURCE_2_ratio_application',
 'Day_Diff_installments_min_sd_installments_payments',
 'Day_Diff_installments_sd_mean_installments_payments',
 'EXT_SOURCE_1_APARTMENTS_AVG_ratio_application',
 'MONTHS_BALANCE_sd_sd_POS_CASH',
 'MONTHS_BALANCE_sum_sd_POS_CASH',
 'NAME_CONTRACT_STATUS_mean_mean_credit_card_balance',
 'NUM_INSTALMENT_NUMBER_mean_min_installments_payments',
 'NUM_INSTALMENT_VERSION_sum_mean_installments_payments',
 'Payment_Diff_installments_max_sd_installments_payments',
 'Payment_Diff_installments_n_distinct_mean_installments_payments',
 'AMT_ANNUITY_CNT_FAM_MEMBERS_ratio_application',
 'AMT_CREDIT_OWN_CAR_AGE_ratio_application',
 'AMT_GOODS_PRICE_DAYS_ID_PUBLISH_ratio_application',
 'AMT_GOODS_PRICE_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'AMT_INCOME_TOTAL_EXT_SOURCE_2_ratio_application',
 'AMT_INSTALMENT_max_mean_installments_payments',
 'AMT_INSTALMENT_max_min_installments_payments',
 'AMT_INSTALMENT_median_sum_installments_payments',
 'AMT_INSTALMENT_min_sum_installments_payments',
 'AMT_PAYMENT_max_mean_installments_payments',
 'AMT_PAYMENT_median_min_installments_payments',
 'AMT_PAYMENT_median_sum_installments_payments',
 'APARTMENTS_AVG_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'CNT_CHILDREN_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'CNT_DRAWINGS_ATM_CURRENT_mean_Mode_credit_card_balance',
 'CNT_INSTALMENT_FUTURE_Mode_Mode_POS_CASH',
 'COMMONAREA_MODE',
 'DAYS_ENTRY_PAYMENT_Mode_min_installments_payments',
 'DAYS_ID_PUBLISH_AMT_INCOME_TOTAL_ratio_application',
 'DAYS_INSTALMENT_min_sd_installments_payments',
 'DAYS_REGISTRATION_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'Day_Diff_installments_mean_sum_installments_payments',
 'Day_Diff_installments_median_max_installments_payments',
 'Day_Diff_installments_min_Mode_installments_payments',
 'Day_Diff_installments_n_distinct_Mode_installments_payments',
 'EXT_SOURCE_1_DAYS_REGISTRATION_ratio_application',
 'EXT_SOURCE_3_DAYS_REGISTRATION_ratio_application',
 'MAX_AMT_CREDIT_prev_app',
 'MAX_Delta_Credit_prev_app',
 'MIN_AMT_ANNUITY_bureau',
 'NAME_CONTRACT_STATUS_mean_sum_POS_CASH',
 'NAME_FAMILY_STATUS_Married',
 'NUM_INSTALMENT_VERSION_mean_sum_installments_payments',
 'ORGANIZATION_TYPE_Business Entity Type 3',
 'SK_DPD_DEF_max_Mode_POS_CASH',
 'SK_DPD_DEF_n_distinct_sum_POS_CASH',
 'SK_DPD_DEF_sd_max_POS_CASH',
 'SK_DPD_DEF_sum_Mode_POS_CASH',
 'SUM_AMT_ANNUITY_prev_app',
 'SUM_AMT_CREDIT_prev_app',
 'THURSDAY_count_WEEKDAY_APPR_PROCESS_START_prev_app',
 'TUESDAY_count_WEEKDAY_APPR_PROCESS_START_prev_app',
 'Time_Since_Status_1_bbalance',
 'Unnamed: 0',
 'XNA_count_NAME_CASH_LOAN_PURPOSE_prev_app',
 'AMT_INSTALMENT_mean_median_installments_payments',
 'AMT_INSTALMENT_median_sd_installments_payments',
 'AMT_INSTALMENT_sum_Mode_installments_payments',
 'AMT_PAYMENT_sum_min_installments_payments',
 'AMT_RECIVABLE_min_Mode_credit_card_balance',
 'APARTMENTS_AVG_DAYS_EMPLOYED_ratio_application',
 'CNT_CHILDREN_DAYS_ID_PUBLISH_ratio_application',
 'CNT_DRAWINGS_ATM_CURRENT_sd_Mode_credit_card_balance',
 'CNT_DRAWINGS_CURRENT_mean_min_credit_card_balance',
 'CNT_INSTALMENT_FUTURE_sd_min_POS_CASH',
 'CNT_INSTALMENT_sum_Mode_POS_CASH',
 'Cash_count_PRODUCT_COMBINATION_prev_app',
 'DAYS_EMPLOYED_DAYS_REGISTRATION_ratio_application',
 'DAYS_ENTRY_PAYMENT_Mode_mean_installments_payments',
 'DAYS_ENTRY_PAYMENT_mean_mean_installments_payments',
 'DAYS_ID_PUBLISH_CNT_FAM_MEMBERS_ratio_application',
 'DAYS_INSTALMENT_min_Mode_installments_payments',
 'DAYS_INSTALMENT_sd_mean_installments_payments',
 'DAYS_INSTALMENT_sd_sum_installments_payments',
 'DAYS_LAST_PHONE_CHANGE_APARTMENTS_AVG_ratio_application',
 'DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio_application',
 'DAYS_LAST_PHONE_CHANGE_OBS_60_CNT_SOCIAL_CIRCLE_ratio_application',
 'DAYS_REGISTRATION_AMT_INCOME_TOTAL_ratio_application',
 'Day_Diff_installments_Mode_sum_installments_payments',
 'Day_Diff_installments_sum_max_installments_payments',
 'EXT_SOURCE_2_APARTMENTS_AVG_ratio_application',
 'LANDAREA_AVG',
 'MEAN_AMT_CREDIT_SUM_LIMIT_bureau',
 'MEAN_AMT_CREDIT_prev_app',
 'MEAN_DAYS_FIRST_DUE_prev_app',
 'MONTHS_BALANCE_sd_sum_POS_CASH',
 'NAME_CONTRACT_STATUS_n_distinct_sd_POS_CASH',
 'NAME_HOUSING_TYPE_With parents',
 'NUM_INSTALMENT_VERSION_max_sd_installments_payments',
 'NUM_INSTALMENT_VERSION_sd_mean_installments_payments',
 'NUM_INSTALMENT_VERSION_sd_min_installments_payments',
 'ORGANIZATION_TYPE_School',
 'ORGANIZATION_TYPE_Self-employed',
 'OWN_CAR_AGE_APARTMENTS_AVG_ratio_application',
 'SK_DPD_mean_median_POS_CASH',
 'SUM_CNT_PAYMENT_prev_app',
 'na',
 'AMT_ANNUITY_DAYS_REGISTRATION_ratio_application',
 'AMT_BALANCE_median_Mode_credit_card_balance',
 'AMT_INCOME_TOTAL_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'AMT_INCOME_TOTAL_OWN_CAR_AGE_ratio_application',
 'AMT_INSTALMENT_Mode_mean_installments_payments',
 'AMT_INSTALMENT_sd_min_installments_payments',
 'AMT_INSTALMENT_sd_sd_installments_payments',
 'AMT_INSTALMENT_sd_sum_installments_payments',
 'AMT_INSTALMENT_sum_max_installments_payments',
 'AMT_PAYMENT_Mode_median_installments_payments',
 'AMT_PAYMENT_max_median_installments_payments',
 'AMT_PAYMENT_mean_min_installments_payments',
 'AMT_PAYMENT_median_sd_installments_payments',
 'AMT_PAYMENT_n_distinct_sum_installments_payments',
 'AMT_RECIVABLE_sd_mean_credit_card_balance',
 'BASEMENTAREA_AVG',
 'CNT_INSTALMENT_FUTURE_n_distinct_max_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sd_mean_POS_CASH',
 'CNT_INSTALMENT_median_median_POS_CASH',
 'CNT_INSTALMENT_sum_sum_POS_CASH',
 'DAYS_ENTRY_PAYMENT_n_distinct_max_installments_payments',
 'DAYS_INSTALMENT_sum_median_installments_payments',
 'Day_Diff_installments_sd_max_installments_payments',
 'Day_Diff_installments_sum_mean_installments_payments',
 'EXT_SOURCE_1_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'EXT_SOURCE_2_DAYS_ID_PUBLISH_ratio_application',
 'HOUR_APPR_PROCESS_START',
 'MAX_SELLERPLACE_AREA_prev_app',
 'MEAN_CNT_PAYMENT_prev_app',
 'MEAN_DAYS_TERMINATION_prev_app',
 'MIN_AMT_CREDIT_prev_app',
 'MIN_DAYS_LAST_DUE_prev_app',
 'MONTHS_BALANCE_min_sum_POS_CASH',
 'MONTHS_BALANCE_n_distinct_max_POS_CASH',
 'NAME_CONTRACT_STATUS_sum_median_POS_CASH',
 'NUM_INSTALMENT_NUMBER_sum_min_installments_payments',
 'NUM_INSTALMENT_VERSION_mean_sd_installments_payments',
 'NUM_INSTALMENT_VERSION_sum_max_installments_payments',
 'OBS_60_CNT_SOCIAL_CIRCLE_DAYS_ID_PUBLISH_ratio_application',
 'Other_count_NAME_CASH_LOAN_PURPOSE_prev_app',
 'POSmobilewithinterest_count_PRODUCT_COMBINATION_prev_app',
 'Payment_Diff_installments_mean_max_installments_payments',
 'REG_CITY_NOT_LIVE_CITY',
 'SD_AMT_ANNUITY_prev_app',
 'SD_DAYS_LAST_DUE_1ST_VERSION_prev_app',
 'SK_DPD_DEF_sum_median_POS_CASH',
 'SUM_AMT_APPLICATION_prev_app',
 'SUM_DAYS_FIRST_DUE_prev_app',
 'Time_Since_Status_0_bbalance',
 'AMT_CREDIT_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'AMT_INSTALMENT_sum_min_installments_payments',
 'AMT_PAYMENT_TOTAL_CURRENT_mean_sum_credit_card_balance',
 'AMT_PAYMENT_n_distinct_min_installments_payments',
 'AMT_TOTAL_RECEIVABLE_max_mean_credit_card_balance',
 'BASEMENTAREA_MODE',
 'CNT_CHILDREN_DAYS_BIRTH_ratio_application',
 'CNT_FAM_MEMBERS_APARTMENTS_AVG_ratio_application',
 'CNT_INSTALMENT_FUTURE_median_min_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sd_max_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sd_sum_POS_CASH',
 'CNT_INSTALMENT_min_sum_POS_CASH',
 'CNT_INSTALMENT_sum_max_POS_CASH',
 'Children_count_NAME_TYPE_SUITE_prev_app',
 'DAYS_ENTRY_PAYMENT_Mode_median_installments_payments',
 'DAYS_ENTRY_PAYMENT_min_max_installments_payments',
 'DAYS_ENTRY_PAYMENT_n_distinct_mean_installments_payments',
 'DAYS_ENTRY_PAYMENT_sd_median_installments_payments',
 'DAYS_INSTALMENT_mean_sd_installments_payments',
 'DAYS_INSTALMENT_sum_sd_installments_payments',
 'DAYS_REGISTRATION_DAYS_BIRTH_ratio_application',
 'DAYS_REGISTRATION_EXT_SOURCE_2_ratio_application',
 'DEF_30_CNT_SOCIAL_CIRCLE_DAYS_ID_PUBLISH_ratio_application',
 'DEF_30_CNT_SOCIAL_CIRCLE_OBS_60_CNT_SOCIAL_CIRCLE_ratio_application',
 'Day_Diff_installments_max_sd_installments_payments',
 'Day_Diff_installments_mean_sd_installments_payments',
 'Day_Diff_installments_sum_sd_installments_payments',
 'EXT_SOURCE_1_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'EXT_SOURCE_range_application',
 'LIVINGAPARTMENTS_AVG',
 'MIN_AMT_APPLICATION_prev_app',
 'MONTHS_BALANCE_Mode_min_POS_CASH',
 'MONTHS_BALANCE_sum_median_POS_CASH',
 'NAME_CONTRACT_STATUS_mean_mean_POS_CASH',
 'NAME_CONTRACT_STATUS_sd_median_POS_CASH',
 'NAME_CONTRACT_STATUS_sum_sd_POS_CASH',
 'NUM_INSTALMENT_NUMBER_sum_sum_installments_payments',
 'OBS_30_CNT_SOCIAL_CIRCLE_DEF_30_CNT_SOCIAL_CIRCLE_ratio_application',
 'Payment_Diff_installments_max_sum_installments_payments',
 'Payment_Diff_installments_sd_median_installments_payments',
 'Payment_Diff_installments_sum_sd_installments_payments',
 'SK_DPD_DEF_sum_mean_POS_CASH',
 'SUM_DAYS_ENDDATE_FACT_bureau',
 'WEEKDAY_APPR_PROCESS_START_MONDAY',
 'XAP_count_CODE_REJECT_REASON_prev_app',
 'YEARS_BUILD_MODE',
 '_count_NAME_TYPE_SUITE_prev_app',
 'AMT_BALANCE_mean_min_credit_card_balance',
 'AMT_BALANCE_median_mean_credit_card_balance',
 'AMT_GOODS_PRICE_OWN_CAR_AGE_ratio_application',
 'AMT_INSTALMENT_max_sd_installments_payments',
 'AMT_INSTALMENT_mean_mean_installments_payments',
 'AMT_PAYMENT_CURRENT_sum_Mode_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_max_mean_credit_card_balance',
 'AMT_PAYMENT_min_sd_installments_payments',
 'AMT_PAYMENT_sd_mean_installments_payments',
 'AMT_PAYMENT_sd_median_installments_payments',
 'AMT_PAYMENT_sum_mean_installments_payments',
 'AMT_RECEIVABLE_PRINCIPAL_min_Mode_credit_card_balance',
 'APARTMENTS_AVG_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'APARTMENTS_AVG_OBS_30_CNT_SOCIAL_CIRCLE_ratio_application',
 'Bureau_Past_Credit_Count',
 'CNT_CHILDREN_AMT_CREDIT_ratio_application',
 'CNT_DRAWINGS_POS_CURRENT_sd_max_credit_card_balance',
 'CNT_INSTALMENT_FUTURE_min_sum_POS_CASH',
 'CNT_INSTALMENT_FUTURE_n_distinct_sum_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sum_mean_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sum_n_distinct_POS_CASH',
 'CNT_INSTALMENT_mean_mean_POS_CASH',
 'CNT_INSTALMENT_min_Mode_POS_CASH',
 'CNT_INSTALMENT_sd_median_POS_CASH',
 'CNT_INSTALMENT_sd_min_POS_CASH',
 'Connectivity_count_NAME_SELLER_INDUSTRY_prev_app',
 'DAYS_BIRTH_APARTMENTS_AVG_ratio_application',
 'DAYS_BIRTH_DAYS_REGISTRATION_ratio_application',
 'DAYS_BIRTH_OBS_30_CNT_SOCIAL_CIRCLE_ratio_application',
 'DAYS_EMPLOYED_OBS_60_CNT_SOCIAL_CIRCLE_ratio_application',
 'DAYS_EMPLOYED_OWN_CAR_AGE_ratio_application',
 'DAYS_ENTRY_PAYMENT_Mode_Mode_installments_payments',
 'DAYS_ENTRY_PAYMENT_Mode_sd_installments_payments',
 'DAYS_ENTRY_PAYMENT_mean_sd_installments_payments',
 'DAYS_ENTRY_PAYMENT_sum_median_installments_payments',
 'DAYS_ID_PUBLISH_OBS_60_CNT_SOCIAL_CIRCLE_ratio_application',
 'DAYS_INSTALMENT_Mode_median_installments_payments',
 'DAYS_INSTALMENT_max_max_installments_payments',
 'DAYS_INSTALMENT_max_median_installments_payments',
 'DAYS_INSTALMENT_min_max_installments_payments',
 'DAYS_INSTALMENT_min_min_installments_payments',
 'DAYS_REGISTRATION_DAYS_ID_PUBLISH_ratio_application',
 'DEF_30_CNT_SOCIAL_CIRCLE_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'DEF_30_CNT_SOCIAL_CIRCLE_DAYS_REGISTRATION_ratio_application',
 'Day_Diff_installments_median_Mode_installments_payments',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MEDI',
 'MEAN_DAYS_LAST_DUE_1ST_VERSION_prev_app',
 'MEAN_DAYS_LAST_DUE_prev_app',
 'MONTHS_BALANCE_max_Mode_POS_CASH',
 'MONTHS_BALANCE_max_mean_POS_CASH',
 'MONTHS_BALANCE_mean_max_POS_CASH',
 'MONTHS_BALANCE_mean_sum_POS_CASH',
 'MONTHS_BALANCE_min_Mode_POS_CASH',
 'MONTHS_BALANCE_sum_Mode_POS_CASH',
 'MONTHS_BALANCE_sum_mean_POS_CASH',
 'NAME_CONTRACT_STATUS_sd_mean_credit_card_balance',
 'NAME_CONTRACT_STATUS_sd_min_POS_CASH',
 'NUM_INSTALMENT_NUMBER_mean_sum_installments_payments',
 'NUM_INSTALMENT_NUMBER_median_median_installments_payments',
 'NUM_INSTALMENT_NUMBER_median_sum_installments_payments',
 'NUM_INSTALMENT_NUMBER_sd_sd_installments_payments',
 'NUM_INSTALMENT_VERSION_n_distinct_sum_installments_payments',
 'NUM_INSTALMENT_VERSION_sum_sum_installments_payments',
 'OBS_30_CNT_SOCIAL_CIRCLE_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'OWN_CAR_AGE_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'Payment_Diff_installments_max_Mode_installments_payments',
 'Payment_Diff_installments_mean_sd_installments_payments',
 'Payment_Diff_installments_median_mean_installments_payments',
 'Payment_Diff_installments_sd_mean_installments_payments',
 'SD_RATE_DOWN_PAYMENT_prev_app',
 'SK_DPD_DEF_sd_Mode_POS_CASH',
 'SK_DPD_sd_min_POS_CASH',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'AMT_CREDIT_APARTMENTS_AVG_ratio_application',
 'AMT_DRAWINGS_ATM_CURRENT_n_distinct_Mode_credit_card_balance',
 'AMT_DRAWINGS_ATM_CURRENT_sd_max_credit_card_balance',
 'AMT_DRAWINGS_CURRENT_mean_min_credit_card_balance',
 'AMT_GOODS_PRICE_AMT_REQ_CREDIT_BUREAU_QRT_ratio_application',
 'AMT_GOODS_PRICE_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'AMT_GOODS_PRICE_APARTMENTS_AVG_ratio_application',
 'AMT_INSTALMENT_Mode_Mode_installments_payments',
 'AMT_INSTALMENT_max_sum_installments_payments',
 'AMT_INSTALMENT_median_min_installments_payments',
 'AMT_PAYMENT_CURRENT_median_Mode_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_Mode_max_credit_card_balance',
 'AMT_PAYMENT_sd_sum_installments_payments',
 'AMT_RECEIVABLE_PRINCIPAL_sd_min_credit_card_balance',
 'AMT_RECIVABLE_max_min_credit_card_balance',
 'AMT_RECIVABLE_median_mean_credit_card_balance',
 'AMT_RECIVABLE_median_min_credit_card_balance',
 'APARTMENTS_AVG_OBS_60_CNT_SOCIAL_CIRCLE_ratio_application',
 'CNT_CHILDREN_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'CNT_CHILDREN_DAYS_REGISTRATION_ratio_application',
 'CNT_INSTALMENT_FUTURE_Mode_sum_POS_CASH',
 'CNT_INSTALMENT_FUTURE_max_median_POS_CASH',
 'CNT_INSTALMENT_FUTURE_n_distinct_median_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sd_Mode_POS_CASH',
 'CNT_INSTALMENT_max_mean_POS_CASH',
 'CNT_INSTALMENT_mean_max_POS_CASH',
 'CNT_INSTALMENT_sum_median_POS_CASH',
 'CashX-Sell:middle_count_PRODUCT_COMBINATION_prev_app',
 'DAYS_EMPLOYED_CNT_FAM_MEMBERS_ratio_application',
 'DAYS_EMPLOYED_DAYS_BIRTH_ratio_application',
 'DAYS_EMPLOYED_DAYS_ID_PUBLISH_ratio_application',
 'DAYS_EMPLOYED_EXT_SOURCE_2_ratio_application',
 'DAYS_ENTRY_PAYMENT_max_median_installments_payments',
 'DAYS_ENTRY_PAYMENT_mean_median_installments_payments',
 'DAYS_ENTRY_PAYMENT_mean_min_installments_payments',
 'DAYS_ENTRY_PAYMENT_median_max_installments_payments',
 'DAYS_ENTRY_PAYMENT_median_min_installments_payments',
 'DAYS_ENTRY_PAYMENT_min_mean_installments_payments',
 'DAYS_ENTRY_PAYMENT_min_min_installments_payments',
 'DAYS_ENTRY_PAYMENT_min_sd_installments_payments',
 'DAYS_ENTRY_PAYMENT_n_distinct_sum_installments_payments',
 'DAYS_ID_PUBLISH_AMT_ANNUITY_ratio_application',
 'DAYS_ID_PUBLISH_DAYS_REGISTRATION_ratio_application',
 'DAYS_INSTALMENT_max_sum_installments_payments',
 'DAYS_INSTALMENT_median_max_installments_payments',
 'DAYS_INSTALMENT_median_min_installments_payments',
 'DAYS_INSTALMENT_median_sd_installments_payments',
 'DAYS_INSTALMENT_n_distinct_max_installments_payments',
 'DAYS_INSTALMENT_sum_Mode_installments_payments',
 'DAYS_LAST_PHONE_CHANGE_CNT_CHILDREN_ratio_application',
 'DAYS_LAST_PHONE_CHANGE_CNT_FAM_MEMBERS_ratio_application',
 'DAYS_REGISTRATION_DEF_30_CNT_SOCIAL_CIRCLE_ratio_application',
 'Day_Diff_installments_Mode_median_installments_payments',
 'Day_Diff_installments_max_max_installments_payments',
 'Day_Diff_installments_max_min_installments_payments',
 'Day_Diff_installments_mean_median_installments_payments',
 'Day_Diff_installments_median_mean_installments_payments',
 'EXT_SOURCE_1_OBS_30_CNT_SOCIAL_CIRCLE_ratio_application',
 'FLAG_EMAIL',
 'MAX_DAYS_LAST_DUE_prev_app',
 'MEAN_CNT_INSTALLMENT_MODE_POS_CASH',
 'MIN_DAYS_LAST_DUE_1ST_VERSION_prev_app',
 'MONTHS_BALANCE_max_max_POS_CASH',
 'MONTHS_BALANCE_max_min_POS_CASH',
 'MONTHS_BALANCE_max_sd_POS_CASH',
 'MONTHS_BALANCE_min_max_POS_CASH',
 'MONTHS_BALANCE_n_distinct_sum_POS_CASH',
 'NAME_CONTRACT_STATUS_max_sd_POS_CASH',
 'NAME_CONTRACT_STATUS_sum_Mode_POS_CASH',
 'NONLIVINGAREA_MEDI',
 'NUM_INSTALMENT_NUMBER_max_max_installments_payments',
 'NUM_INSTALMENT_NUMBER_max_sd_installments_payments',
 'NUM_INSTALMENT_NUMBER_sd_max_installments_payments',
 'NUM_INSTALMENT_NUMBER_sum_sd_installments_payments',
 'NUM_INSTALMENT_VERSION_median_sd_installments_payments',
 'OBS_30_CNT_SOCIAL_CIRCLE_DAYS_EMPLOYED_ratio_application',
 'Payment_Diff_installments_max_max_installments_payments',
 'Payment_Diff_installments_n_distinct_sum_installments_payments',
 'Repeater_count_NAME_CLIENT_TYPE_prev_app',
 'SK_DPD_DEF_sd_sd_POS_CASH',
 'SK_DPD_mean_sum_POS_CASH',
 'SK_DPD_sd_median_POS_CASH',
 'SUM_DAYS_LAST_DUE_prev_app',
 'SUM_DAYS_TERMINATION_prev_app',
 'Time_Since_Status_C_bbalance',
 'WEDNESDAY_count_WEEKDAY_APPR_PROCESS_START_prev_app',
 'YEARS_BEGINEXPLUATATION_MODE',
 'prev_app_count',
 'AMT_ANNUITY_OBS_30_CNT_SOCIAL_CIRCLE_ratio_application',
 'AMT_DRAWINGS_ATM_CURRENT_sd_min_credit_card_balance',
 'AMT_GOODS_PRICE_OBS_60_CNT_SOCIAL_CIRCLE_ratio_application',
 'AMT_INCOME_TOTAL',
 'AMT_INCOME_TOTAL_APARTMENTS_AVG_ratio_application',
 'AMT_INSTALMENT_Mode_max_installments_payments',
 'AMT_INSTALMENT_max_median_installments_payments',
 'AMT_INSTALMENT_mean_max_installments_payments',
 'AMT_INSTALMENT_median_Mode_installments_payments',
 'AMT_INSTALMENT_median_median_installments_payments',
 'AMT_INSTALMENT_sd_max_installments_payments',
 'AMT_INSTALMENT_sd_mean_installments_payments',
 'AMT_INST_MIN_REGULARITY_mean_mean_credit_card_balance',
 'AMT_INST_MIN_REGULARITY_mean_min_credit_card_balance',
 'AMT_INST_MIN_REGULARITY_sd_mean_credit_card_balance',
 'AMT_PAYMENT_CURRENT_mean_min_credit_card_balance',
 'AMT_PAYMENT_CURRENT_sd_Mode_credit_card_balance',
 'AMT_PAYMENT_Mode_Mode_installments_payments',
 'AMT_PAYMENT_Mode_mean_installments_payments',
 'AMT_PAYMENT_Mode_min_installments_payments',
 'AMT_PAYMENT_Mode_sd_installments_payments',
 'AMT_PAYMENT_Mode_sum_installments_payments',
 'AMT_PAYMENT_TOTAL_CURRENT_mean_Mode_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_mean_max_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_n_distinct_mean_credit_card_balance',
 'AMT_PAYMENT_max_max_installments_payments',
 'AMT_PAYMENT_mean_max_installments_payments',
 'AMT_PAYMENT_mean_mean_installments_payments',
 'AMT_PAYMENT_mean_sd_installments_payments',
 'AMT_PAYMENT_mean_sum_installments_payments',
 'AMT_PAYMENT_n_distinct_max_installments_payments',
 'AMT_PAYMENT_sd_max_installments_payments',
 'AMT_RECIVABLE_n_distinct_min_credit_card_balance',
 'AMT_RECIVABLE_sum_mean_credit_card_balance',
 'AMT_REQ_CREDIT_BUREAU_QRT_DAYS_ID_PUBLISH_ratio_application',
 'APARTMENTS_AVG_DAYS_ID_PUBLISH_ratio_application',
 'Approved_count_NAME_CONTRACT_STATUS_prev_app',
 'BASEMENTAREA_MEDI',
 'CNT_CHILDREN_AMT_GOODS_PRICE_ratio_application',
 'CNT_CHILDREN_APARTMENTS_AVG_ratio_application',
 'CNT_DRAWINGS_CURRENT_mean_Mode_credit_card_balance',
 'CNT_FAM_MEMBERS_DAYS_ID_PUBLISH_ratio_application',
 'CNT_INSTALMENT_FUTURE_max_max_POS_CASH',
 'CNT_INSTALMENT_FUTURE_max_sd_POS_CASH',
 'CNT_INSTALMENT_FUTURE_max_sum_POS_CASH',
 'CNT_INSTALMENT_FUTURE_n_distinct_min_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sum_Mode_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sum_max_POS_CASH',
 'CardStreet_count_PRODUCT_COMBINATION_prev_app',
 'Cashloans_count_NAME_CONTRACT_TYPE_prev_app',
 'Country-wide_count_CHANNEL_TYPE_prev_app',
 'DAYS_BIRTH_DAYS_EMPLOYED_ratio_application',
 'DAYS_BIRTH_OBS_60_CNT_SOCIAL_CIRCLE_ratio_application',
 'DAYS_EMPLOYED_AMT_CREDIT_ratio_application',
 'DAYS_EMPLOYED_CNT_CHILDREN_ratio_application',
 'DAYS_EMPLOYED_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'DAYS_ENTRY_PAYMENT_Mode_sum_installments_payments',
 'DAYS_ENTRY_PAYMENT_max_mean_installments_payments',
 'DAYS_ENTRY_PAYMENT_median_mean_installments_payments',
 'DAYS_ENTRY_PAYMENT_min_median_installments_payments',
 'DAYS_ENTRY_PAYMENT_sd_mean_installments_payments',
 'DAYS_ENTRY_PAYMENT_sd_sd_installments_payments',
 'DAYS_ENTRY_PAYMENT_sum_sd_installments_payments',
 'DAYS_INSTALMENT_max_Mode_installments_payments',
 'DAYS_INSTALMENT_max_sd_installments_payments',
 'DAYS_INSTALMENT_mean_sum_installments_payments',
 'DAYS_INSTALMENT_min_median_installments_payments',
 'DAYS_INSTALMENT_sd_max_installments_payments',
 'DAYS_REGISTRATION_CNT_FAM_MEMBERS_ratio_application',
 'Day_Diff_installments_Mode_Mode_installments_payments',
 'Day_Diff_installments_max_mean_installments_payments',
 'Day_Diff_installments_max_median_installments_payments',
 'Day_Diff_installments_max_sum_installments_payments',
 'Day_Diff_installments_n_distinct_sd_installments_payments',
 'ENTRANCES_AVG',
 'ENTRANCES_MEDI',
 'EXT_SOURCE_1_OBS_60_CNT_SOCIAL_CIRCLE_ratio_application',
 'EXT_SOURCE_2_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'INC_PER_CHLD',
 'LANDAREA_MODE',
 'MAX_AMT_ANNUITY_prev_app',
 'MAX_DAYS_TERMINATION_prev_app',
 'MIN_AMT_CREDIT_MAX_OVERDUE_bureau',
 'MIN_RATE_DOWN_PAYMENT_prev_app',
 'MONTHS_BALANCE_Mode_mean_POS_CASH',
 'MONTHS_BALANCE_Mode_sum_POS_CASH',
 'MONTHS_BALANCE_max_sum_POS_CASH',
 'MONTHS_BALANCE_min_mean_POS_CASH',
 'MONTHS_BALANCE_sd_max_POS_CASH',
 'NAME_CONTRACT_STATUS_mean_median_POS_CASH',
 'NAME_CONTRACT_STATUS_mean_sd_POS_CASH',
 'NAME_CONTRACT_STATUS_min_sum_POS_CASH',
 'NAME_CONTRACT_STATUS_sd_sum_POS_CASH',
 'NAME_CONTRACT_STATUS_sum_min_POS_CASH',
 'NAME_HOUSING_TYPE_House / apartment',
 'NUM_INSTALMENT_NUMBER_Mode_mean_installments_payments',
 'NUM_INSTALMENT_NUMBER_Mode_sd_installments_payments',
 'NUM_INSTALMENT_NUMBER_max_mean_installments_payments',
 'NUM_INSTALMENT_NUMBER_sd_min_installments_payments',
 'NUM_INSTALMENT_NUMBER_sd_sum_installments_payments',
 'NUM_INSTALMENT_VERSION_Mode_sd_installments_payments',
 'NUM_INSTALMENT_VERSION_max_sum_installments_payments',
 'NUM_INSTALMENT_VERSION_mean_median_installments_payments',
 'NUM_INSTALMENT_VERSION_n_distinct_mean_installments_payments',
 'NUM_INSTALMENT_VERSION_sd_Mode_installments_payments',
 'NUM_INSTALMENT_VERSION_sum_Mode_installments_payments',
 'OBS_30_CNT_SOCIAL_CIRCLE_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'OBS_60_CNT_SOCIAL_CIRCLE_DAYS_LAST_PHONE_CHANGE_ratio_application',
 'OCCUPATION_TYPE_Laborers',
 'OWN_CAR_AGE_DAYS_ID_PUBLISH_ratio_application',
 'OWN_CAR_AGE_OBS_60_CNT_SOCIAL_CIRCLE_ratio_application',
 'Payment_Diff_installments_sd_max_installments_payments',
 'Payment_Diff_installments_sd_sd_installments_payments',
 'Payment_Diff_installments_sum_min_installments_payments',
 'REGION_POPULATION_RELATIVE',
 'SD_AMT_CREDIT_SUM_LIMIT_bureau',
 'SD_CNT_INSTALLMENT_MODE_POS_CASH',
 'SK_DPD_DEF_sum_sd_POS_CASH',
 'SK_DPD_sum_median_POS_CASH',
 'WALLSMATERIAL_MODE_Panel',
 'XNA_count_NAME_PAYMENT_TYPE_prev_app',
 'YEARS_BUILD_MEDI',
 'currency1_count_CREDIT_CURRENCY_bureau',
 '1_count_bbalance',
 'AMT_BALANCE_median_max_credit_card_balance',
 'AMT_BALANCE_sd_min_credit_card_balance',
 'AMT_BALANCE_sum_min_credit_card_balance',
 'AMT_CREDIT_LIMIT_ACTUAL_mean_Mode_credit_card_balance',
 'AMT_CREDIT_LIMIT_ACTUAL_mean_max_credit_card_balance',
 'AMT_CREDIT_LIMIT_ACTUAL_mean_sum_credit_card_balance',
 'AMT_CREDIT_LIMIT_ACTUAL_median_max_credit_card_balance',
 'AMT_CREDIT_LIMIT_ACTUAL_sum_Mode_credit_card_balance',
 'AMT_DRAWINGS_ATM_CURRENT_mean_min_credit_card_balance',
 'AMT_DRAWINGS_CURRENT_mean_Mode_credit_card_balance',
 'AMT_DRAWINGS_CURRENT_mean_max_credit_card_balance',
 'AMT_DRAWINGS_CURRENT_sd_sum_credit_card_balance',
 'AMT_DRAWINGS_POS_CURRENT_mean_Mode_credit_card_balance',
 'AMT_DRAWINGS_POS_CURRENT_sum_max_credit_card_balance',
 'AMT_INSTALMENT_Mode_median_installments_payments',
 'AMT_INSTALMENT_Mode_min_installments_payments',
 'AMT_INSTALMENT_max_max_installments_payments',
 'AMT_INSTALMENT_mean_sd_installments_payments',
 'AMT_INSTALMENT_mean_sum_installments_payments',
 'AMT_INSTALMENT_n_distinct_mean_installments_payments',
 'AMT_INSTALMENT_n_distinct_median_installments_payments',
 'AMT_INSTALMENT_n_distinct_sd_installments_payments',
 'AMT_INST_MIN_REGULARITY_max_Mode_credit_card_balance',
 'AMT_INST_MIN_REGULARITY_max_mean_credit_card_balance',
 'AMT_INST_MIN_REGULARITY_max_sum_credit_card_balance',
 'AMT_PAYMENT_CURRENT_mean_sum_credit_card_balance',
 'AMT_PAYMENT_CURRENT_median_min_credit_card_balance',
 'AMT_PAYMENT_CURRENT_sum_max_credit_card_balance',
 'AMT_PAYMENT_CURRENT_sum_min_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_Mode_sum_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_max_Mode_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_mean_mean_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_mean_min_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_median_sum_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_sd_Mode_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_sd_min_credit_card_balance',
 'AMT_PAYMENT_TOTAL_CURRENT_sum_Mode_credit_card_balance',
 'AMT_PAYMENT_max_sum_installments_payments',
 'AMT_RECEIVABLE_PRINCIPAL_max_min_credit_card_balance',
 'AMT_RECEIVABLE_PRINCIPAL_mean_Mode_credit_card_balance',
 'AMT_RECEIVABLE_PRINCIPAL_mean_mean_credit_card_balance',
 'AMT_RECEIVABLE_PRINCIPAL_median_min_credit_card_balance',
 'AMT_RECIVABLE_max_max_credit_card_balance',
 'AMT_RECIVABLE_max_sum_credit_card_balance',
 'AMT_RECIVABLE_mean_max_credit_card_balance',
 'AMT_RECIVABLE_median_Mode_credit_card_balance',
 'AMT_RECIVABLE_sd_max_credit_card_balance',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_QRT_DAYS_REGISTRATION_ratio_application',
 'AMT_REQ_CREDIT_BUREAU_YEAR_DAYS_REGISTRATION_ratio_application',
 'AMT_TOTAL_RECEIVABLE_sd_min_credit_card_balance',
 'AP+(Cashloan)_count_CHANNEL_TYPE_prev_app',
 'APARTMENTS_AVG',
 'APARTMENTS_AVG_DAYS_REGISTRATION_ratio_application',
 'CAR_TO_BIRTH_RATIO',
 'CNT_DRAWINGS_ATM_CURRENT_n_distinct_sum_credit_card_balance',
 'CNT_DRAWINGS_CURRENT_mean_sum_credit_card_balance',
 'CNT_DRAWINGS_CURRENT_median_min_credit_card_balance',
 'CNT_DRAWINGS_CURRENT_sd_Mode_credit_card_balance',
 'CNT_DRAWINGS_CURRENT_sd_mean_credit_card_balance',
 'CNT_DRAWINGS_CURRENT_sd_sum_credit_card_balance',
 'CNT_DRAWINGS_CURRENT_sum_max_credit_card_balance',
 'CNT_DRAWINGS_CURRENT_sum_min_credit_card_balance',
 'CNT_DRAWINGS_POS_CURRENT_max_mean_credit_card_balance',
 'CNT_FAM_MEMBERS',
 'CNT_FAM_MEMBERS_AMT_REQ_CREDIT_BUREAU_YEAR_ratio_application',
 'CNT_FAM_MEMBERS_DAYS_REGISTRATION_ratio_application',
 'CNT_INSTALMENT_FUTURE_max_Mode_POS_CASH',
 'CNT_INSTALMENT_FUTURE_max_min_POS_CASH',
 'CNT_INSTALMENT_FUTURE_sum_sum_POS_CASH',
 'CNT_INSTALMENT_MATURE_CUM_max_max_credit_card_balance',
 'CNT_INSTALMENT_MATURE_CUM_sd_Mode_credit_card_balance',
 'CNT_INSTALMENT_max_Mode_POS_CASH',
 'CNT_INSTALMENT_max_sd_POS_CASH',
 'CNT_INSTALMENT_min_median_POS_CASH',
 'CNT_INSTALMENT_n_distinct_Mode_POS_CASH',
 'CNT_INSTALMENT_n_distinct_n_distinct_POS_CASH',
 'Canceled_count_NAME_CONTRACT_STATUS_prev_app',
 'Consumerelectronics_count_NAME_SELLER_INDUSTRY_prev_app',
 'DAYS_EMPLOYED_AMT_GOODS_PRICE_ratio_application',
 'DAYS_ENTRY_PAYMENT_max_Mode_installments_payments',
 'DAYS_ENTRY_PAYMENT_max_min_installments_payments',
 'DAYS_ENTRY_PAYMENT_max_sum_installments_payments',
 'DAYS_ENTRY_PAYMENT_mean_Mode_installments_payments',
 'DAYS_ENTRY_PAYMENT_mean_max_installments_payments',
 'DAYS_ENTRY_PAYMENT_mean_sum_installments_payments',
 'DAYS_ENTRY_PAYMENT_median_median_installments_payments',
 'DAYS_ENTRY_PAYMENT_min_Mode_installments_payments',
 'DAYS_ENTRY_PAYMENT_min_sum_installments_payments',
 'DAYS_ENTRY_PAYMENT_n_distinct_median_installments_payments',
 'DAYS_ENTRY_PAYMENT_sum_sum_installments_payments',
 'DAYS_INSTALMENT_Mode_max_installments_payments',
 'DAYS_INSTALMENT_Mode_mean_installments_payments',
 'DAYS_INSTALMENT_mean_median_installments_payments']

In [11]:
#select top features
application_train = application_train.loc[:,Extracted_features + ['SK_ID_CURR','TARGET', 'fold', 'Weights']]
application_test = application_test.loc[:, Extracted_features + ['SK_ID_CURR']]

In [12]:
train_cols = [i for i in application_train.columns]
test_cols = [i for i in application_test.columns]
train_not_test = [i for i in train_cols if i not in test_cols]
test_not_train = [i for i in test_cols if i not in train_cols]

#The following must be true going forward:
assert(train_not_test == ['TARGET', 'fold', 'Weights'])
assert(test_not_train == [])
print("Train and Test columns are as expected!")

Train and Test columns are as expected!


In [13]:
Train_DMatrix = list()
Validate_DMatrix = list()

for i in range(0,5):
    print(i)
    Train_tmp = application_train.loc[application_train.fold != i+1,: ]
    Validate_tmp = application_train.loc[application_train.fold == i+1, :]
    
    Train_DMatrix.append( 
    xgb.DMatrix(
        data  = Train_tmp.drop(columns = ["SK_ID_CURR","Weights", "TARGET", "fold"]), 
        weight= Train_tmp.Weights,
        label = Train_tmp.TARGET))
    
    Validate_DMatrix.append( 
    xgb.DMatrix(
        data  = Validate_tmp.drop(columns = ["SK_ID_CURR","Weights", "TARGET", "fold"]), 
        weight= Validate_tmp.Weights,
        label = Validate_tmp.TARGET))
    



0
1
2
3
4


### 5-Fold CV

In [ ]:
Trained_Models = list()

for i in range(0,5):
    print("Training on fold: " + str(i))
    watchlist = [(Validate_DMatrix[i], 'eval'), (Train_DMatrix[i], 'train')]
    Trained_Models.append(xgb.train(params=par, 
                                    num_boost_round=rounds,
                                    dtrain=Train_DMatrix[i],
                                    evals=watchlist))


Training on fold: 0
[0]	eval-auc:0.538758	train-auc:0.555464
[1]	eval-auc:0.538758	train-auc:0.555464
[2]	eval-auc:0.541355	train-auc:0.559721
[3]	eval-auc:0.541363	train-auc:0.559741
[4]	eval-auc:0.588373	train-auc:0.593665
[5]	eval-auc:0.588543	train-auc:0.594024
[6]	eval-auc:0.58869	train-auc:0.594408
[7]	eval-auc:0.588723	train-auc:0.594476
[8]	eval-auc:0.588735	train-auc:0.59451
[9]	eval-auc:0.588901	train-auc:0.594667
[10]	eval-auc:0.592373	train-auc:0.599528
[11]	eval-auc:0.5924	train-auc:0.599612
[12]	eval-auc:0.592459	train-auc:0.599672
[13]	eval-auc:0.596066	train-auc:0.60809
[14]	eval-auc:0.596127	train-auc:0.609017
[15]	eval-auc:0.603359	train-auc:0.615872
[16]	eval-auc:0.605323	train-auc:0.616736
[17]	eval-auc:0.605361	train-auc:0.616891
[18]	eval-auc:0.605306	train-auc:0.616978
[19]	eval-auc:0.614664	train-auc:0.632428
[20]	eval-auc:0.616298	train-auc:0.63545
[21]	eval-auc:0.616337	train-auc:0.635592
[22]	eval-auc:0.618257	train-auc:0.635483
[23]	eval-auc:0.620172	train-a

[194]	eval-auc:0.723202	train-auc:0.77495
[195]	eval-auc:0.723504	train-auc:0.775101
[196]	eval-auc:0.723682	train-auc:0.775462
[197]	eval-auc:0.723944	train-auc:0.775712
[198]	eval-auc:0.724047	train-auc:0.775985
[199]	eval-auc:0.724174	train-auc:0.77613
[200]	eval-auc:0.724168	train-auc:0.776271
[201]	eval-auc:0.72422	train-auc:0.776541
[202]	eval-auc:0.724331	train-auc:0.776705
[203]	eval-auc:0.724506	train-auc:0.77699
[204]	eval-auc:0.724863	train-auc:0.777362
[205]	eval-auc:0.725044	train-auc:0.777565
[206]	eval-auc:0.72527	train-auc:0.777973
[207]	eval-auc:0.725538	train-auc:0.77832
[208]	eval-auc:0.72569	train-auc:0.778635
[209]	eval-auc:0.725994	train-auc:0.779069
[210]	eval-auc:0.72653	train-auc:0.77981
[211]	eval-auc:0.726636	train-auc:0.780223
[212]	eval-auc:0.726801	train-auc:0.780568
[213]	eval-auc:0.727099	train-auc:0.78104
[214]	eval-auc:0.727263	train-auc:0.781402
[215]	eval-auc:0.727628	train-auc:0.781992
[216]	eval-auc:0.727895	train-auc:0.782451
[217]	eval-auc:0.7280

[386]	eval-auc:0.755784	train-auc:0.834979
[387]	eval-auc:0.755875	train-auc:0.835248
[388]	eval-auc:0.755969	train-auc:0.835599
[389]	eval-auc:0.756129	train-auc:0.835877
[390]	eval-auc:0.756193	train-auc:0.83611
[391]	eval-auc:0.756401	train-auc:0.83631
[392]	eval-auc:0.756496	train-auc:0.836564
[393]	eval-auc:0.756526	train-auc:0.836829
[394]	eval-auc:0.756615	train-auc:0.837122
[395]	eval-auc:0.756799	train-auc:0.837409
[396]	eval-auc:0.756971	train-auc:0.837655
[397]	eval-auc:0.75714	train-auc:0.83795
[398]	eval-auc:0.757244	train-auc:0.838195
[399]	eval-auc:0.75737	train-auc:0.838423
[400]	eval-auc:0.757554	train-auc:0.838734
[401]	eval-auc:0.757635	train-auc:0.838994
[402]	eval-auc:0.757753	train-auc:0.83922
[403]	eval-auc:0.757705	train-auc:0.839433
[404]	eval-auc:0.757856	train-auc:0.839654
[405]	eval-auc:0.758009	train-auc:0.83986
[406]	eval-auc:0.758128	train-auc:0.840074
[407]	eval-auc:0.75825	train-auc:0.840344
[408]	eval-auc:0.75835	train-auc:0.840609
[409]	eval-auc:0.758

[578]	eval-auc:0.769507	train-auc:0.871582
[579]	eval-auc:0.769622	train-auc:0.871691
[580]	eval-auc:0.769665	train-auc:0.871809
[581]	eval-auc:0.769688	train-auc:0.871931
[582]	eval-auc:0.769723	train-auc:0.872076
[583]	eval-auc:0.769733	train-auc:0.872194
[584]	eval-auc:0.769813	train-auc:0.872333
[585]	eval-auc:0.769806	train-auc:0.872489
[586]	eval-auc:0.769836	train-auc:0.872575
[587]	eval-auc:0.769944	train-auc:0.872704
[588]	eval-auc:0.769939	train-auc:0.872828
[589]	eval-auc:0.769996	train-auc:0.872982
[590]	eval-auc:0.770011	train-auc:0.873071
[591]	eval-auc:0.770018	train-auc:0.873219
[592]	eval-auc:0.770034	train-auc:0.873303
[593]	eval-auc:0.770079	train-auc:0.873396
[594]	eval-auc:0.770089	train-auc:0.873482
[595]	eval-auc:0.770025	train-auc:0.873571
[596]	eval-auc:0.769963	train-auc:0.873637
[597]	eval-auc:0.769973	train-auc:0.873735
[598]	eval-auc:0.770037	train-auc:0.873904
[599]	eval-auc:0.770053	train-auc:0.874021
[600]	eval-auc:0.770226	train-auc:0.874113
[601]	eval-

[770]	eval-auc:0.773973	train-auc:0.889031
[771]	eval-auc:0.77399	train-auc:0.889108
[772]	eval-auc:0.77397	train-auc:0.889214
[773]	eval-auc:0.773968	train-auc:0.889262
[774]	eval-auc:0.773952	train-auc:0.889284
[775]	eval-auc:0.773982	train-auc:0.889396
[776]	eval-auc:0.773986	train-auc:0.889474
[777]	eval-auc:0.773948	train-auc:0.889564
[778]	eval-auc:0.774031	train-auc:0.889637
[779]	eval-auc:0.774048	train-auc:0.889709
[780]	eval-auc:0.774012	train-auc:0.889775
[781]	eval-auc:0.774097	train-auc:0.889829
[782]	eval-auc:0.774128	train-auc:0.889885
[783]	eval-auc:0.774198	train-auc:0.889956
[784]	eval-auc:0.774221	train-auc:0.889971
[785]	eval-auc:0.774314	train-auc:0.890081
[786]	eval-auc:0.774313	train-auc:0.890143
[787]	eval-auc:0.774302	train-auc:0.890227
[788]	eval-auc:0.774298	train-auc:0.890259
[789]	eval-auc:0.774297	train-auc:0.890297
[790]	eval-auc:0.774378	train-auc:0.890379
[791]	eval-auc:0.774414	train-auc:0.890443
[792]	eval-auc:0.774483	train-auc:0.890482
[793]	eval-au

[962]	eval-auc:0.775874	train-auc:0.898296
[963]	eval-auc:0.775918	train-auc:0.898331
[964]	eval-auc:0.775907	train-auc:0.898412
[965]	eval-auc:0.775916	train-auc:0.898432
[966]	eval-auc:0.775961	train-auc:0.898472
[967]	eval-auc:0.775935	train-auc:0.898533
[968]	eval-auc:0.775935	train-auc:0.898533
[969]	eval-auc:0.775958	train-auc:0.898583
[970]	eval-auc:0.775963	train-auc:0.898622
[971]	eval-auc:0.775977	train-auc:0.898639
[972]	eval-auc:0.77603	train-auc:0.898697
[973]	eval-auc:0.77604	train-auc:0.89872
[974]	eval-auc:0.776026	train-auc:0.898765
[975]	eval-auc:0.776018	train-auc:0.89885
[976]	eval-auc:0.776019	train-auc:0.898887
[977]	eval-auc:0.776124	train-auc:0.898928
[978]	eval-auc:0.776085	train-auc:0.898972
[979]	eval-auc:0.776078	train-auc:0.899002
[980]	eval-auc:0.776073	train-auc:0.899054
[981]	eval-auc:0.776073	train-auc:0.899054
[982]	eval-auc:0.77608	train-auc:0.899087
[983]	eval-auc:0.776099	train-auc:0.899124
[984]	eval-auc:0.776131	train-auc:0.899162
[985]	eval-auc:0

[1151]	eval-auc:0.776413	train-auc:0.90465
[1152]	eval-auc:0.776393	train-auc:0.904667
[1153]	eval-auc:0.776423	train-auc:0.904727
[1154]	eval-auc:0.776453	train-auc:0.90477
[1155]	eval-auc:0.776494	train-auc:0.904845
[1156]	eval-auc:0.776494	train-auc:0.904845
[1157]	eval-auc:0.776489	train-auc:0.904917
[1158]	eval-auc:0.776481	train-auc:0.904926
[1159]	eval-auc:0.776478	train-auc:0.904957
[1160]	eval-auc:0.776457	train-auc:0.905025
[1161]	eval-auc:0.776467	train-auc:0.905054
[1162]	eval-auc:0.776462	train-auc:0.905064
[1163]	eval-auc:0.776459	train-auc:0.905111
[1164]	eval-auc:0.776483	train-auc:0.905147
[1165]	eval-auc:0.776485	train-auc:0.905182
[1166]	eval-auc:0.776488	train-auc:0.905208
[1167]	eval-auc:0.776492	train-auc:0.905285
[1168]	eval-auc:0.776492	train-auc:0.905285
[1169]	eval-auc:0.776488	train-auc:0.905317
[1170]	eval-auc:0.77648	train-auc:0.905328
[1171]	eval-auc:0.776469	train-auc:0.905341
[1172]	eval-auc:0.776469	train-auc:0.905341
[1173]	eval-auc:0.776419	train-auc:

[1339]	eval-auc:0.776665	train-auc:0.909282
[1340]	eval-auc:0.77665	train-auc:0.909297
[1341]	eval-auc:0.77665	train-auc:0.909297
[1342]	eval-auc:0.77665	train-auc:0.909297
[1343]	eval-auc:0.776684	train-auc:0.909312
[1344]	eval-auc:0.776684	train-auc:0.909312
[1345]	eval-auc:0.776671	train-auc:0.909346
[1346]	eval-auc:0.776671	train-auc:0.909346
[1347]	eval-auc:0.776671	train-auc:0.909346
[1348]	eval-auc:0.776678	train-auc:0.909372
[1349]	eval-auc:0.776678	train-auc:0.909372
[1350]	eval-auc:0.776652	train-auc:0.909401
[1351]	eval-auc:0.77665	train-auc:0.909419
[1352]	eval-auc:0.77665	train-auc:0.909419
[1353]	eval-auc:0.77665	train-auc:0.909419
[1354]	eval-auc:0.7767	train-auc:0.909437
[1355]	eval-auc:0.7767	train-auc:0.909437
[1356]	eval-auc:0.776712	train-auc:0.909465
[1357]	eval-auc:0.776712	train-auc:0.909465
[1358]	eval-auc:0.776712	train-auc:0.909465
[1359]	eval-auc:0.776719	train-auc:0.909495
[1360]	eval-auc:0.776719	train-auc:0.909495
[1361]	eval-auc:0.776719	train-auc:0.90949

[1527]	eval-auc:0.776987	train-auc:0.912054
[1528]	eval-auc:0.776983	train-auc:0.912073
[1529]	eval-auc:0.776983	train-auc:0.912073
[1530]	eval-auc:0.777011	train-auc:0.912127
[1531]	eval-auc:0.777018	train-auc:0.912165
[1532]	eval-auc:0.777045	train-auc:0.912213
[1533]	eval-auc:0.777024	train-auc:0.912225
[1534]	eval-auc:0.777024	train-auc:0.912225
[1535]	eval-auc:0.777024	train-auc:0.912225
[1536]	eval-auc:0.77706	train-auc:0.912251
[1537]	eval-auc:0.777046	train-auc:0.91227
[1538]	eval-auc:0.777046	train-auc:0.91227
[1539]	eval-auc:0.777046	train-auc:0.91227
[1540]	eval-auc:0.777086	train-auc:0.912306
[1541]	eval-auc:0.777086	train-auc:0.912306
[1542]	eval-auc:0.777099	train-auc:0.912354
[1543]	eval-auc:0.777099	train-auc:0.912354
[1544]	eval-auc:0.777099	train-auc:0.912354
[1545]	eval-auc:0.777099	train-auc:0.912354
[1546]	eval-auc:0.777099	train-auc:0.912354
[1547]	eval-auc:0.777081	train-auc:0.912385
[1548]	eval-auc:0.777091	train-auc:0.912397
[1549]	eval-auc:0.777091	train-auc:0

Training/Validation Output for log:

In [ ]:
log_title = "Log_" + run_title + ".txt"
f = open(log_title, "w")

f.write("----------Training Results---------------- \n")
for i in range(0,5):
    tmp = Trained_Models[i].eval(Train_DMatrix[i])
    f.write("Fold " + str(i+1) + tmp)
    f.write("\n")
f.write("----------Training Results----------------\n")
for i in range(0,5):
    tmp = Trained_Models[i].eval(Validate_DMatrix[i])
    f.write("Fold " + str(i+1) + tmp)
    f.write("\n")
f.close()

### Calculated weighted AUC on each fold

In [ ]:
Weighted_AUCs_Valid = list()
Weighted_AUCs_Train = list()
for i in range(0,5):
    tmp_predict = Trained_Models[i].predict(data = Validate_DMatrix[i])
    tmp_auc = roc_auc_score(y_true = 
                            application_train.loc[application_train.fold == (i+1),:].TARGET,
                           y_score = tmp_predict,
                           sample_weight = application_train.loc[application_train.fold == (i+1),:].Weights)
    Weighted_AUCs_Valid.append(tmp_auc)
    
    
for i in range(0,5):
    tmp_predict = Trained_Models[i].predict(data = Train_DMatrix[i])
    tmp_auc = roc_auc_score(y_true = 
                            application_train.loc[application_train.fold != (i+1),:].TARGET,
                           y_score = tmp_predict,
                           sample_weight = application_train.loc[application_train.fold != (i+1),:].Weights)
    Weighted_AUCs_Train.append(tmp_auc)

### Log 5-fold results, save as csv

In [ ]:
fold_names = ['fold_1', 'fold_2', 'fold_3', 'fold_4', 'fold_5', 'fold_AVG']
Weighted_AUCs_Train.append( sum(Weighted_AUCs_Train) / 5)
Weighted_AUCs_Valid.append( sum(Weighted_AUCs_Valid) / 5)
Validation_Log = pd.DataFrame()
Validation_Log['Fold_Name'] = pd.Series(fold_names)
Validation_Log['Validation_AUC'] = pd.Series(Weighted_AUCs_Valid)
Validation_Log['Train_AUC'] = pd.Series(Weighted_AUCs_Train)
Validation_Log.to_csv(output_path + "Log_"+run_title + '.csv')

In [ ]:
del Trained_Models
del Train_DMatrix
del Validate_DMatrix
gc.collect()

### Train Model

In [ ]:
Train_DMatrix_All = xgb.DMatrix(
        data  = application_train.drop(columns = ["SK_ID_CURR","Weights","fold", "TARGET"]), 
        weight= application_train.Weights,
        label = application_train.TARGET)

Test_DMatrix_All = xgb.DMatrix(
    data = application_test.drop(columns = ["SK_ID_CURR"]))

In [ ]:
watchlist = [(Train_DMatrix_All, 'train')]

Model = xgb.train(par, Train_DMatrix_All, rounds, watchlist)

In [ ]:
Test_Predictions = Model.predict(data=Test_DMatrix_All)

In [ ]:
Submission = pd.DataFrame(application_test.loc[:,"SK_ID_CURR"])
Submission["TARGET"] = Test_Predictions
Submission_Final = pd.merge(sample_submission.drop(columns = ["TARGET"]), 
                            Submission, how = "left",on = "SK_ID_CURR")

In [ ]:
Submission_Final.to_csv(output_path + Output_Name,
                       index = False)

In [ ]:
import subprocess
print(subprocess.check_output(['./kaggle_submit.sh', Output_Name, "Notes"]))

In [ ]:
subprocess.call(['./shutdown_script.sh'])